## Even Odd with Serverless Framework

The original example can be found at: https://github.com/rosafilgueira/dispel4py_training_material/blob/master/dispel4py-tutorial/dispel4py_example_EvenOdd/EvenOddworkflow.ipynb

This example uses the serverless framework to execute the workflow given above and provides alternative methods of execution

Prior set-up

In [ ]:
#Auto-load and set directory
%load_ext autoreload
%autoreload 2

import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

Import serverless client module along with required dispel4py modules

In [ ]:
from dispel4py.base import *
from dispel4py.workflow_graph import WorkflowGraph
from client import d4pClient
import random

Create PEs 

In [ ]:
class NumberProducer(ProducerPE):
    def __init__(self):
        ProducerPE.__init__(self)
        
    def _process(self , inputs):
        result= random.randint(1, 1000)
        return result
        #OR: self.write('output', result)

class Divideby2(IterativePE):

    def __init__(self, compare):
        IterativePE.__init__(self)
        self.compare = compare

    def _process(self, data):
        if data % 2 == self.compare:
            return data

class PairProducer(GenericPE):

    def __init__(self):
        GenericPE.__init__(self)
        self._add_input("odd")
        self._add_input("even")
        self._add_output("output")
        self.list_odd=[]
        self.list_even=[]

    def _process(self, inputs):
        if "odd" in inputs:
            self.list_odd.append(inputs["odd"])
        if "even" in inputs:
            self.list_even.append(inputs["even"])
       
        while self.list_odd and self.list_even:
            self.write("output", (self.list_odd.pop(0), self.list_even.pop(0)))
    
    def _postprocess(self):
        self.log('We are left behind: odd: %s, even: %s' % (self.list_odd, self.list_even))
        self.list_odd = []
        self.list_even = []

#### Option 1: Direct Execution

Create PE object from each class 

In [ ]:
client = d4pClient()

producer = NumberProducer()
filter_even = Divideby2(0)
filter_odd = Divideby2(1)
pair = PairProducer()

Create workflow from PEs

In [ ]:
graph = WorkflowGraph()
graph.connect(producer, 'output', filter_even, 'input')
graph.connect(producer, 'output', filter_odd, 'input')
graph.connect(filter_even, 'output', pair, 'even')
graph.connect(filter_odd, 'output', pair, 'odd')

Execute workflow directly

In [ ]:
result = client.run(graph,input=20)

print(result)

#### Option 2: Use Registry

Register PEs

In [ ]:
pe_1 = client.register_PE(producer)
pe_2 = client.register_PE(filter_even)
pe_3 = client.register_PE(pair)

Retrieve PE from Registry

In [ ]:
producer_pe = client.get_PE(pe_1)
filter_pe = client.get_PE(pe_2)
pair_pe = client.get_PE(pe_3)

Construct workflow from PEs

In [ ]:
#Set compare variable for DivideBy2 PE 
filter_even_pe = filter_pe
filter_even_pe.compare = 0

filter_odd_pe = filter_pe
filter_odd_pe.compare = 1

graph = WorkflowGraph()
graph.connect(producer_pe, 'output', filter_even_pe, 'input')
graph.connect(producer_pe, 'output', filter_odd_pe, 'input')
graph.connect(filter_even_pe, 'output', pair_pe, 'even')
graph.connect(filter_odd_pe, 'output', pair_pe, 'odd')

Execute workflow

In [ ]:
result = client.run(graph,input=20)

print(result)